In [1]:
from logging import getLogger
import os
import json
import pandas as pd
import time, datetime

from recbole.model.general_recommender.lightgcn import LightGCN

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_logger, get_trainer, init_seed, set_color
from recbole.quick_start.quick_start import load_data_and_model

from recbole.config import Config
from recbole.data import create_dataset

from sklearn.metrics import accuracy_score, roc_auc_score

import torch

In [2]:
data = pd.read_csv('data/train/train_ratings.csv')

In [8]:
item_set = set(data['item'].unique().tolist())
for i in data['user'].unique():
    if i in item_set:
        print("duplicated")
        break

duplicated


In [2]:
data = pd.read_csv('data/train/train_ratings.csv')
data['answer'] = 1
data['item'] = 'i' + data['item'].astype(str)
print(data['user'].nunique())
print(data['item'].nunique())
print(data.shape)
data.head()

31360
6807
(5154471, 4)


,user,item,time,answer
0,11,i4643,1230782529,1
1,11,i170,1230782534,1
2,11,i531,1230782539,1
3,11,i616,1230782542,1
4,11,i2140,1230782563,1


In [79]:
userid, itemid = sorted(list(set(data.user))), sorted(list(set(data.item)))
n_user, n_item = len(userid), len(itemid)
userid_2_index = {v:i for i, v in enumerate(userid)}
itemid_2_index = {v:i+n_user for i, v in enumerate(itemid)}
id_2_index = dict(userid_2_index, **itemid_2_index)
index_2_id = {i: v for v, i in id_2_index.items()}
index_2_userid = {i: v for v, i in userid_2_index.items()}
index_2_itemid = {i: v for v, i in itemid_2_index.items()}

In [85]:
id_2_index['i10']

31361

In [82]:
itemid_2_index

{'i1': 31360,
 'i10': 31361,
 'i100': 31362,
 'i100108': 31363,
 'i100163': 31364,
 'i100244': 31365,
 'i100302': 31366,
 'i100344': 31367,
 'i100383': 31368,
 'i100390': 31369,
 'i100487': 31370,
 'i100498': 31371,
 'i1005': 31372,
 'i100527': 31373,
 'i100556': 31374,
 'i1006': 31375,
 'i1007': 31376,
 'i100714': 31377,
 'i100745': 31378,
 'i1009': 31379,
 'i101': 31380,
 'i1010': 31381,
 'i101025': 31382,
 'i101088': 31383,
 'i1011': 31384,
 'i101106': 31385,
 'i101112': 31386,
 'i101142': 31387,
 'i1012': 31388,
 'i101285': 31389,
 'i1013': 31390,
 'i101360': 31391,
 'i101362': 31392,
 'i1014': 31393,
 'i1015': 31394,
 'i101525': 31395,
 'i101577': 31396,
 'i1016': 31397,
 'i1017': 31398,
 'i101739': 31399,
 'i101741': 31400,
 'i1018': 31401,
 'i101864': 31402,
 'i101895': 31403,
 'i1019': 31404,
 'i101962': 31405,
 'i101973': 31406,
 'i1020': 31407,
 'i102033': 31408,
 'i1021': 31409,
 'i102123': 31410,
 'i102125': 31411,
 'i102194': 31412,
 'i1022': 31413,
 'i1023': 31414,
 'i102

In [4]:
yamldata = """
USER_ID_FIELD: user_id
ITEM_ID_FIELD: item_id
RATING_FIELD: rating
TIME_FIELD: timestamp

load_col:
    inter: [user_id, item_id, rating, timestamp]

user_inter_num_interval: "[0,inf)"
item_inter_num_interval: "[0,inf)"
val_interval:
    rating: "[0,1]"
    timestamp: "[97830000, inf)"
"""

In [50]:
outpath = f"dataset/train_data"
outfile = f"dataset/train_data/train_data.inter"
yamlfile = f"train_data.yaml"

os.makedirs(outpath, exist_ok=True)

print("Processing Start")
inter_table = []
for user, item, acode, tstamp in zip(data.user, data.item, data.answer, data.time):
    uid, iid = id_2_index[user], id_2_index[item]
    tval = tstamp
    inter_table.append( [uid, iid, max(acode,0), tval] )

print("Processing Complete")

print("Dump Start")
# 데이터 설정 파일 저장
with open(yamlfile, "w") as f:
    f.write(yamldata) 

# 데이터 파일 저장
with open(outfile, "w") as f:
    # write header
    f.write("user_id:token\titem_id:token\trating:float\ttimestamp:float\n")
    for row in inter_table:
        f.write("\t".join([str(x) for x in row])+"\n")

print("Dump Complete")

Processing Start
Processing Complete
Dump Start
Dump Complete


In [27]:
# outpath = f"dataset/test_data"
# outfile = f"dataset/test_data/test_data.inter"
# yamlfile = f"test_data.yaml"

# os.makedirs(outpath, exist_ok=True)

# print("Processing Start")
# inter_table = []
# for user, item, acode, tstamp in zip(eval_data.user, eval_data.item, eval_data.answer, eval_data.time):
#     uid, iid = id_2_index[user], id_2_index[item]
#     tval = tstamp
#     inter_table.append( [uid, iid, max(acode,0), tval] )

# print("Processing Complete")

# print("Dump Start")
# # 데이터 설정 파일 저장
# with open(yamlfile, "w") as f:
#     f.write(yamldata) 

# # 데이터 파일 저장
# with open(outfile, "w") as f:
#     # write header
#     f.write("user_id:token\titem_id:token\trating:float\ttimestamp:float\n")
#     for row in inter_table:
#         f.write("\t".join([str(x) for x in row])+"\n")

# print("Dump Complete")

Processing Start
Processing Complete
Dump Start
Dump Complete


In [5]:
logger = getLogger()

In [5]:
# configurations initialization
config = Config(model='LightGCN', dataset="train_data", config_file_list=[f'train_data.yaml'])
config['epochs'] = 50
config['show_progress'] = False
config['device'] = 'cuda' if torch.cuda.is_available() else "cpu"
config['device'] = torch.device(config['device'])
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)

logger.info(config)

21 Jun 13:23    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 50
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_s

In [6]:
# dataset filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

21 Jun 13:24    INFO  train_data
The number of users: 31361
Average actions of users: 164.36450892857144
The number of items: 6808
Average actions of items: 757.2309387395328
The number of inters: 5154471
The sparsity of the dataset: 97.58579218741939%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']
21 Jun 13:24    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
21 Jun 13:24    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


In [7]:
# model loading and initialization
init_seed(config['seed'], config['reproducibility'])
model = LightGCN(config, train_data.dataset).to(config['device'])
logger.info(model)

21 Jun 13:24    INFO  LightGCN(
  (user_embedding): Embedding(31361, 64)
  (item_embedding): Embedding(6808, 64)
  (mf_loss): BPRLoss()
  (reg_loss): EmbLoss()
)
Trainable parameters: 2442816


In [8]:
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
trainer.device

device(type='cuda')

In [116]:
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(
    train_data, valid_data, saved=True, show_progress=config['show_progress']
)

21 Jun 07:31    INFO  epoch 0 training [time: 144.69s, train loss: 659.1427]
21 Jun 07:31    INFO  epoch 0 evaluating [time: 24.48s, valid_score: 0.277100]
21 Jun 07:31    INFO  valid result: 
recall@10 : 0.081    mrr@10 : 0.2771    ndcg@10 : 0.1253    hit@10 : 0.5749    precision@10 : 0.1041
21 Jun 07:31    INFO  Saving current: saved/LightGCN-Jun-21-2023_07-28-54.pth
21 Jun 07:34    INFO  epoch 1 training [time: 144.66s, train loss: 586.7964]
21 Jun 07:34    INFO  epoch 1 evaluating [time: 27.48s, valid_score: 0.284600]
21 Jun 07:34    INFO  valid result: 
recall@10 : 0.0842    mrr@10 : 0.2846    ndcg@10 : 0.1304    hit@10 : 0.5895    precision@10 : 0.1093
21 Jun 07:34    INFO  Saving current: saved/LightGCN-Jun-21-2023_07-28-54.pth
21 Jun 07:37    INFO  epoch 2 training [time: 144.81s, train loss: 521.6588]
21 Jun 07:37    INFO  epoch 2 evaluating [time: 24.44s, valid_score: 0.302500]
21 Jun 07:37    INFO  valid result: 
recall@10 : 0.0912    mrr@10 : 0.3025    ndcg@10 : 0.1408    h

In [ ]:
# model evaluation
test_result = trainer.evaluate(test_data, load_best_model="True", show_progress=config['show_progress'])

logger.info(set_color('best valid ', 'yellow') + f': {best_valid_result}')
logger.info(set_color('test result', 'yellow') + f': {test_result}')

result = {
    'best_valid_score': best_valid_score,
    'valid_score_bigger': config['valid_metric_bigger'],
    'best_valid_result': best_valid_result,
    'test_result': test_result
}

print(json.dumps(result, indent=4))

In [92]:
# configurations initialization
config = Config(model='LightGCN', dataset="test_data", config_file_list=[f'test_data.yaml'])
config['epochs'] = 1
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)

# dataset filtering
test_dataset = create_dataset(config)
logger.info(test_dataset)
test_dataset
# # 성능 측정
user1 = model.full_sort_predict(test_dataset)

# a_prob = model.predict(test_dataset).tolist()
# print(a_prob)
# a_true = [val for val in test_dataset.inter_feat["rating"]]
# print(a_true)
# a_pred = [round(v) for v in a_prob]
# print(a_pred)


21 Jun 13:58    INFO  test_data
The number of users: 31361
Average actions of users: 1.0
The number of items: 2
Average actions of items: 31360.0
The number of inters: 31360
The sparsity of the dataset: 50.001594336915275%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']


In [6]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='saved/LightGCN-Jun-21-2023_07-28-54.pth',
)

22 Jun 05:12    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 50
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_s

In [16]:
# configurations initialization
config = Config(model='LightGCN', dataset="test_data", config_file_list=[f'test_data.yaml'])
config['epochs'] = 1
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)

# dataset filtering
test_dataset = create_dataset(config)
logger.info(test_dataset)

# # 성능 측정
user1 = model.full_sort_predict(test_dataset)

21 Jun 13:32    INFO  test_data
The number of users: 2
Average actions of users: 1.0
The number of items: 2
Average actions of items: 1.0
The number of inters: 1
The sparsity of the dataset: 75.0%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']


In [14]:
item_data = pd.DataFrame()
item_data['item'] = itemid

In [86]:
test_data = pd.DataFrame(columns=['user', 'item', 'answer', 'time'])

# for user in range(n_user):
user_df = pd.DataFrame({'user':[0] * n_item, 'item':itemid, 'answer':0, 'time':1230782529})
test_data = pd.concat([test_data, user_df])
test_data

,user,item,answer,time
0,0,i1,0,1230782529
1,0,i10,0,1230782529
2,0,i100,0,1230782529
3,0,i100108,0,1230782529
4,0,i100163,0,1230782529
...,...,...,...,...
6802,0,i99813,0,1230782529
6803,0,i999,0,1230782529
6804,0,i99910,0,1230782529
6805,0,i99912,0,1230782529


In [90]:
outpath = f"dataset/test_data"
outfile = f"dataset/test_data/test_data.inter"
yamlfile = f"test_data.yaml"

os.makedirs(outpath, exist_ok=True)

print("Processing Start")
inter_table = []
# for user, item, acode, tstamp in zip(test_data.user, test_data.item, test_data.answer, test_data.time):
for item in itemid:
    uid, iid = id_2_index[11], id_2_index['i1']
    tval = tstamp
    inter_table.append( [uid, iid, max(acode,0), tval] )

print("Processing Complete")

print("Dump Start")
# 데이터 설정 파일 저장
with open(yamlfile, "w") as f:
    f.write(yamldata) 

# 데이터 파일 저장
with open(outfile, "w") as f:
    # write header
    f.write("user_id:token\titem_id:token\trating:float\ttimestamp:float\n")
    for row in inter_table:
        f.write("\t".join([str(x) for x in row])+"\n")

print("Dump Complete")

Processing Start
Processing Complete
Dump Start
Dump Complete


In [92]:
import numpy as np
# configurations initialization
config = Config(model='LightGCN', dataset="test_data", config_file_list=[f'test_data.yaml'])
config['epochs'] = 1
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)

# dataset filtering
test_dataset = create_dataset(config)
logger.info(test_dataset)

model.predict(test_dataset)
# # # 성능 측정
# predict = model.full_sort_predict(test_dataset)
# predict = predict.view(-1, n_item + 1)
# predict[:, 0] = -np.inf  # set scores of [pad] to -inf

22 Jun 06:55    INFO  test_data
The number of users: 2
Average actions of users: 1.0
The number of items: 2
Average actions of items: 1.0
The number of inters: 1
The sparsity of the dataset: 75.0%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']


tensor([6.3027], device='cuda:0', grad_fn=<SumBackward1>)

In [55]:
import pandas as pd

# Assuming you have a list of user_ids
user_ids = data['user'].unique().tolist()  # Replace [...] with your list of user_ids

# Create an empty DataFrame to store the results
top_items_df = pd.DataFrame(columns=['user', 'item'])

# Iterate over each user_id
for user in range(n_user):
    sorted_tensor, indices = torch.sort(predict[user])
    top_10_indices = indices[-10:]
    top_10_indices = top_10_indices.tolist()
    top_10_indices = [index_2_itemid[i + n_user + 1][1:] for i in top_10_indices]
    user = index_2_userid[user]
    
    # Append the user_id and top item names to the DataFrame
    user_df = pd.DataFrame({'user': user, 'item': top_10_indices})
    top_items_df = pd.concat([top_items_df, user_df], ignore_index=True)

# Print the resulting DataFrame
print(top_items_df)


          user    item
0           11  104243
1           11  102753
2           11  113275
3           11    1047
4           11    1223
...        ...     ...
313595  138493    1130
313596  138493    1032
313597  138493    1284
313598  138493    1271
313599  138493    1283

[313600 rows x 2 columns]


In [57]:
top_items_df

,user,item
0,11,104243
1,11,102753
2,11,113275
3,11,1047
4,11,1223
...,...,...
313595,138493,1130
313596,138493,1032
313597,138493,1284
313598,138493,1271


In [56]:
for i in top_items_df[top_items_df['user']==11]['item'].values:
    print(i in data[data['user']==11]['item'].apply(lambda x: x[1:]))

False
False
False
False
False
False
False
False
False
False


In [166]:
import pandas as pd

top_items_df.to_csv('first.csv', index=False)

In [217]:
from collections import defaultdict
from tqdm import tqdm
import pandas as pd

input_frame = []
total_list=[]

# 앙상블할 파일 이름
weights = [4, 6]
paths = ['ease_300_10_9.csv', 'seed42_SASRec.csv']

# 앙상블할 파일들 input_frame에 넣기
for file_path in paths:
    file = pd.read_csv(file_path)
    grouped = file.groupby("user")["item"].apply(list)
    input_frame.append(grouped)

for user in tqdm(input_frame[0].keys()):
    item_duplicate_check = list()
    for i, weight in zip(range(len(input_frame)), weights):
        output = input_frame[i]
        count = 0
        for content in output[user]:
            if count == weight:
                break
            if content in item_duplicate_check:
                continue
            item_duplicate_check.append(content)
            count += 1
    item_duplicate_check = item_duplicate_check
    for item_index in item_duplicate_check:
        total_list.append([user, item_index])
    
pd.DataFrame(total_list, columns=['user', 'item']).to_csv('ensemble_4ease_6sas.csv', index=False)

100%|██████████| 31360/31360 [00:01<00:00, 23372.85it/s]


In [171]:
weight = [0.6, 0.4]
input_frame = pd.DataFrame()
total_list=[]
paths = ['ease_300_10_9.csv', 'first.csv']
for file_path in paths:
    file = pd.read_csv(file_path)
    input_frame = pd.concat([input_frame, file])
input_frame

,user,item
0,11,4370
1,11,4886
2,11,47
3,11,40815
4,11,32587
...,...,...
313595,138493,112940
313596,138493,103141
313597,138493,1283
313598,138493,1270


In [188]:
weights = [6, 4]
paths = ['ease_300_10_9.csv', 'first.csv']

# 모든 한 데이터 프레임에 csv파일 붙이기
df = pd.read_csv(paths[0]) 
for path_idx in range(1, len(paths)):
    df_be_concated = pd.read_csv(paths[path_idx])
    df = pd.concat([df, df_be_concated['item']], axis=1)

for user in df['user'].unique():
    item_list = set()
    print(df[df['user'] == user]['item'])
    break
    
    
for weight in weights:
    
    pass

    item    item
0   4370  104241
1   4886  102720
2     47  113225
3  40815    1046
4  32587    1222
5   7373    1059
6   8961  114060
7   7438  104211
8      2  108729
9  33004  103141
